In [1]:
# Clone the repository
cd /cluster/scratch/$USER
git clone --recurse-submodules https://github.com/jurgjn/batch-infer.git
cd batch-infer

Cloning into 'batch-infer'...
remote: Enumerating objects: 1261, done.        
remote: Counting objects: 100% (213/213), done.        
remote: Compressing objects: 100% (118/118), done.        
remote: Total 1261 (delta 140), reused 162 (delta 95), pack-reused 1048 (from 1)        
Receiving objects: 100% (1261/1261), 599.19 KiB | 1.20 MiB/s, done.
Resolving deltas: 100% (731/731), done.
Submodule 'batch-infer/software/smk-simple-slurm-eu' (https://github.com/jurgjn/smk-simple-slurm-eu.git) registered for path 'software/smk-simple-slurm-eu'
Cloning into '/cluster/scratch/jjaenes/batch-infer/software/smk-simple-slurm-eu'...
remote: Enumerating objects: 569, done.        
remote: Counting objects: 100% (138/138), done.        
remote: Compressing objects: 100% (53/53), done.        
remote: Total 569 (delta 106), reused 96 (delta 85), pack-reused 431 (from 1)        
Receiving objects: 100% (569/569), 96.98 KiB | 232.00 KiB/s, done.
Resolving deltas: 100% (287/287), done.
Submodule path 

In [3]:
# Directory results/alphafold3_adhoc_examples contains:
# - subdirectory alphafold3_jsons with the input .json files
# - config.yaml with the location of the model weights
ls -1 results/alphafold3_adhoc_examples/alphafold3_jsons/*

results/alphafold3_adhoc_examples/alphafold3_jsons/atox1.json
results/alphafold3_adhoc_examples/alphafold3_jsons/dusp6.json
results/alphafold3_adhoc_examples/alphafold3_jsons/example_2pv7.json
results/alphafold3_adhoc_examples/alphafold3_jsons/mapk1_dusp6.json
results/alphafold3_adhoc_examples/alphafold3_jsons/mapk1.json


In [ ]:
# Every input .json has a 'name' field which has to match the file name.
# It's also restricted to lowercase ASCII, numbers, underscore (_), hyphen (-) and dot (.) characters
# https://github.com/google-deepmind/alphafold3/blob/main/src/alphafold3/common/folding_input.py#L919-L923
cat results/alphafold3_adhoc_examples/alphafold3_jsons/example_2pv7.json

{
  "name": "example_2pv7",
  "sequences": [
    {
      "protein": {
        "id": ["A", "B"],
        "sequence": "GMRESYANENQFGFKTINSDIHKIVIVGGYGKLGGLFARYLRASGYPISILDREDWAVAESILANADVVIVSVPINLTLETIERLKPYLTENMLLADLTSVKREPLAKMLEVHTGAVLGLHPMFGADIASMAKQVVVRCDGRFPERYEWLLEQIQIWGAKIYQTNATEHDHNMTYIQALRHFSTFANGLHLSKQPINLANLLALSSPIYRLELAMIGRLFAQDAELYADIIMDKSENLAVIETLKQTYDEALTFFENNDRQGFIDAFHKVRDWFGDYSEQFLKESRQLLQQANDLKQG"
      }
    }
  ],
  "modelSeeds": [1],
  "dialect": "alphafold3",
  "version": 1
}


In [5]:
# Edit config.yaml to specify location of model weights
cat results/alphafold3_adhoc_examples/config.yaml

# Edit to specify location of your AF3 model parameters (example location below will not work)
alphafold3_models: /cluster/work/beltrao/jjaenes/24.11.13_af3_models


In [38]:
# For additional options such as the AF3 docker image, see the defaults in workflow/config/defaults.yaml
head -n 5 workflow/config/defaults.yaml


alphafold3_databases: /cluster/project/alphafold/alphafold3
#alphafold3_databases: /cluster/work/beltrao/jjaenes/24.11.11_af3_databases
alphafold3_docker: /cluster/project/beltrao/shared/alphafold3/images/alphafold3-v3.0.1.sif



In [14]:
# Start the pipeline with the following arguments
# - `alphafold3_onegpu` target runs data pipeline steps as separate jobs, and then all structure predictions as a single GPU job
# - `results/alphafold3_adhoc_examples` specifies the directory with inputs (alphafold3_jsons), and the config file (config.yaml)
./batch-infer alphafold3_onegpu results/alphafold3_adhoc_examples | sbatch

Submitted batch job 33649572


In [15]:
# This will start the main snakemake job (batch-infer), and one MSA job per input json (alphafold3_msas)
squeue --format="%.18i %.12P %.40j %.8T %.16M %.16l %40R" | column -t --table-right 1,5,6

   JOBID  PARTITION    NAME                                      STATE    TIME  TIME_LIMIT  NODELIST(REASON)
33649359  normal.120h  batch-infer:alphafold3_onegpu=results-al  RUNNING  3:38  7-00:00:00  eu-g5-048-2
33649365  normal.4h    alphafold3_msas:id=mapk1                  RUNNING  3:24     4:00:00  eu-g5-023-1
33649364  normal.4h    alphafold3_msas:id=dusp6                  RUNNING  3:25     4:00:00  eu-g5-021-1
33649363  normal.4h    alphafold3_msas:id=mapk1_dusp6            RUNNING  3:26     4:00:00  eu-g5-017-1
33649362  normal.4h    alphafold3_msas:id=example_2pv7           RUNNING  3:28     4:00:00  eu-g5-015-4
33649361  normal.4h    alphafold3_msas:id=atox1                  RUNNING  3:29     4:00:00  eu-g5-007-4


In [ ]:
# Data pipeline output appears under the alphafold3_msas-subdirectory:
ls -1 results/alphafold3_adhoc_examples/alphafold3_msas

atox1_data.json.gz
dusp6_data.json.gz
example_2pv7_data.json.gz


In [50]:
# After all MSAs have finished the structure prediction job (alphafold3_predictions) will start
squeue --format="%.18i %.12P %.40j %.8T %.16M %.16l %40R" | column -t --table-right 1,5,6

   JOBID  PARTITION    NAME                                      STATE       TIME  TIME_LIMIT  NODELIST(REASON)
33664955  gpupr.120h   alphafold3_predictions:                   PENDING     0:00  3-00:00:00  (Resources)
33649359  normal.120h  batch-infer:alphafold3_onegpu=results-al  RUNNING  1:33:02  7-00:00:00  eu-g5-048-2


In [57]:
# Structure prediction appears under the alphafold3_predictions-subdirectory: 
ls -1 results/alphafold3_adhoc_examples/alphafold3_predictions

atox1
dusp6
example_2pv7
mapk1
mapk1_dusp6


In [58]:
ls -1 results/alphafold3_adhoc_examples/alphafold3_predictions/atox1

atox1_confidences.json.gz
atox1_data.json.gz
atox1_model.cif.gz
atox1_summary_confidences.json.gz
ranking_scores.csv.gz
seed-1_sample-0
seed-1_sample-1
seed-1_sample-2
seed-1_sample-3
seed-1_sample-4
TERMS_OF_USE.md.gz


In [54]:
# Runtimes for some MSAs - dusp6, mapk1, interaction (mapk1_dusp6)
cat results/alphafold3_adhoc_examples/.snakemake-eu/myjobs/33649364.txt | grep "Wall-clock"
cat results/alphafold3_adhoc_examples/.snakemake-eu/myjobs/33649365.txt | grep "Wall-clock"
cat results/alphafold3_adhoc_examples/.snakemake-eu/myjobs/33649363.txt | grep "Wall-clock"

 Wall-clock                      : 00:28:15
 Wall-clock                      : 01:01:34
 Wall-clock                      : 01:31:51


In [56]:
# Runtime for structure prediction
cat results/alphafold3_adhoc_examples/.snakemake-eu/myjobs/33664955.txt | grep "Wall-clock"

 Wall-clock                      : 00:05:18
